## NSMC(Naver Sentiment Movie Corpis
네이버 영화 리뷰 말뭉치<br>
from Korpora import Korpora


In [2]:
from dataclasses import dataclass

@dataclass
class ClassificationTrainArguments:
    pretrained_model_name: str
    downstream_corpus_name: str
    downstream_corpus_root_dir: str
    downstream_model_dir: str
    learning_rate: float
    batch_size: int

args = ClassificationTrainArguments(
    pretrained_model_name="beomi/kcbert-base",#사전 학습된 모델, Hugging Face의 모델 허브
    downstream_corpus_name="nsmc", #네이버 corpus 다운
    downstream_corpus_root_dir="./data",
    downstream_model_dir="./model",
    learning_rate=5e-5,
    batch_size=32
)

In [4]:
from Korpora import Korpora

Korpora.fetch(
    corpus_name=args.downstream_corpus_name,
    root_dir=args.downstream_corpus_root_dir,
    force_download=True,
)

[nsmc] download ratings_train.txt: 14.6MB [00:00, 47.4MB/s]                                                            
[nsmc] download ratings_test.txt: 4.90MB [00:00, 28.9MB/s]                                                             


In [5]:
#kcbert-base 모델 준비
from transformers import BertConfig, BertForSequenceClassification
pretrained_model_config = BertConfig.from_pretrained(
    args.pretrained_model_name,
    num_labels=2,
)
model = BertForSequenceClassification.from_pretrained(
    args.pretrained_model_name,
    config=pretrained_model_config,
)

ModuleNotFoundError: No module named 'transformers'

### Tokenizer
토큰화 수행 프로그램<br>
kcbert-base 모델

In [6]:
#kcbert-base 토크나이저 준비
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(
    args.pretrained_model_name,
    do_lower_case=False, #대소문자 유지, True면 모든 입력 소문자로 변환
) 

ModuleNotFoundError: No module named 'transformers'

# Pytorch's Data Loader

* 파이토치로 딥러닝 모델을 만들려면 반드시 정의해야 한다.
* 데이터를 배치(batch)단위로 모델에 밀어 넣어주는 역할
* 전체 데이터 가운데 일부 인스턴스를 뽑아 배치를 구성
* 데이터셋은 데이터 로더의 구성 요소 중 하나
* 데이터셋은 여러 인스턴스를 보유

데이터 로더 > 데이터셋 > 인스턴스

* batch는 그 모양이 고정적이어야 할 때가 많다. -> 문장들의 토큰(input_ids) 개수가 같아야 한다.

그래서 batch의 shape을 동일하게 만들어 주는 과정을 collate라고 한다.

### Collate
* list -> pytorch의 tensor로 변환
* batch size 통일

In [7]:
import os
# NSMC 데이터 다운로드 및 로드
data_dir = "./data/nsmc"  # 데이터 저장 경로
os.makedirs(data_dir, exist_ok=True)

train_file = f"{data_dir}/ratings_train.txt"
test_file = f"{data_dir}/ratings_test.txt"

if not os.path.exists(train_file):
    from Korpora import Korpora
    Korpora.fetch("nsmc", root_dir=data_dir)



In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer
import pandas as pd

# ✅ 1. 토크나이저 로드
tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")

# ✅ 2. 데이터 불러오기 (필요하면 샘플링하여 테스트)
train_df = pd.read_csv(train_file, sep="\t").dropna().sample(5000)  # 일부만 사용
test_df = pd.read_csv(test_file, sep="\t").dropna().sample(1000)

# ✅ 3. 최적화된 Dataset (미리 토큰화 적용)
class NsmcDataset(Dataset):
    def __init__(self, df, tokenizer, max_length=128):
        self.tokenizer = tokenizer
        self.texts = df["document"].astype(str).tolist()
        self.labels = df["label"].tolist()

        # 🔥 미리 한 번에 토큰화하여 저장
        self.encodings = self.tokenizer(
            self.texts,
            padding="max_length",
            truncation=True,
            max_length=max_length,
            return_tensors="pt"
        )

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return {
            "input_ids": self.encodings["input_ids"][idx],
            "attention_mask": self.encodings["attention_mask"][idx],
            "label": torch.tensor(self.labels[idx], dtype=torch.long),
        }

# ✅ 4. 데이터셋 생성
train_dataset = NsmcDataset(train_df, tokenizer)
test_dataset = NsmcDataset(test_df, tokenizer)

# ✅ 5. 최적화된 DataLoader 설정
train_dataloader = DataLoader(
    train_dataset,
    batch_size=32,
    shuffle=True,  # RandomSampler 제거
    drop_last=False,
    num_workers=0,  # 테스트 결과에 따라 조절 (4~6 추천)
    pin_memory=True,  # GPU 속도 향상
)

test_dataloader = DataLoader(
    test_dataset,
    batch_size=32,
    shuffle=False,
    drop_last=False,
    num_workers=0,
)

# ✅ 6. 속도 테스트
import time
start_time = time.time()
sample = next(iter(train_dataloader))
print(f"로딩 시간: {time.time() - start_time:.2f}초")

# ✅ 데이터 샘플 확인
print(sample["input_ids"].shape)  # torch.Size([32, 128])
print(sample["label"])  # tensor([0, 1, 1, ...])

### Pytorch Lightning
https://minjoo-happy-blog.tistory.com/140

In [ ]:
import torch
import torch.nn.functional as F
import pytorch_lightning as pl
from transformers import AdamW

# PyTorch Lightning을 사용한 감성 분석 모델 정의
class SentimentClassificationTask(pl.LightningModule):
    def __init__(self, model, learning_rate=5e-5):
        super().__init__()
        self.model = model
        self.learning_rate = learning_rate

    def forward(self, input_ids, attention_mask):
        return self.model(input_ids, attention_mask=attention_mask)

    def training_step(self, batch, batch_idx):
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        labels = batch["label"]

        outputs = self(input_ids, attention_mask)
        logits = outputs.logits
        loss = F.cross_entropy(logits, labels)

        self.log("train_loss", loss, prog_bar=True, logger=True)
        return loss

    def configure_optimizers(self):
        return AdamW(self.parameters(), lr=self.learning_rate)

# 모델 및 학습 설정
task = SentimentClassificationTask(model, learning_rate=args.learning_rate)

# PyTorch Lightning Trainer 설정
trainer = pl.Trainer(
    max_epochs=3,  # 학습 횟수
    accelerator="gpu" if torch.cuda.is_available() else "cpu",
    log_every_n_steps=10
)

# 모델 학습 실행
trainer.fit(
    task,
    train_dataloaders=train_dataloader,
)